In [1]:
import os
os.environ['HTTP_PROXY'] = "http://127.0.0.1:10809"
os.environ['HTTPS_PROXY'] = "http://127.0.0.1:10809"

In [2]:
import geemap
import ee
Map=geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [5]:
# # 大庆市
# roi = ee.Geometry.Rectangle([123.9355, 45.8362,125.3055, 46.8762])
# # 天津市
# roi = ee.Geometry.Rectangle([115.9244, 38.6272,117.4944,39.6372])
# # 武汉市
# roi = ee.Geometry.Rectangle([113.7393, 29.8642,115.0993, 30.9242])
# # 珠三角
# roi = ee.Geometry.Rectangle([112.7614, 22.2347,114.0514, 23.2547])
# 青藏高原
roi = ee.Geometry.Rectangle([89.8826, 34.6579,91.3626, 35.8279])
Map.addLayer(roi, {}, "roi")
Map.centerObject(roi,7)

In [6]:
# 定义参数
region = 'prd'

In [7]:
# 移除无效像元
elevation = ee.Image("users/311605001111/hillshade_" + region)
def maskSR(img):
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5)
    snowBitMask = (1 << 4)   
    qa = img.select('pixel_qa')
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
                   .And(qa.bitwiseAnd(cloudsBitMask).eq(0)) \
                   .And(qa.bitwiseAnd(snowBitMask).eq(0))
    azimuth = img.get('SOLAR_AZIMUTH_ANGLE')
    zenith = img.get('SOLAR_ZENITH_ANGLE')
    image = img.lt(0)
    bands = image.select('B2').add(image.select('B3')).add(image.select('B4')).add(image.select('B5')).add(image.select('B6')).add(image.select('B7'))
    outlier = bands.gt(0).remap([0,1],[1,0]).rename('outlier')
    return img.updateMask(mask).updateMask(ee.Terrain.hillShadow(elevation,azimuth,zenith,200,True)).updateMask(outlier)
def maskSR_reverse(img):
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5)
    snowBitMask = (1 << 4)   
    qa = img.select('pixel_qa')
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
                   .And(qa.bitwiseAnd(cloudsBitMask).eq(0)) \
                   .And(qa.bitwiseAnd(snowBitMask).eq(0))
    image_cloud = img.updateMask(mask.remap([0,1],[1,0]))
    azimuth = img.get('SOLAR_AZIMUTH_ANGLE')
    zenith = img.get('SOLAR_ZENITH_ANGLE')
    image_shadow = img.updateMask(ee.Terrain.hillShadow(elevation,azimuth,zenith,200,True).remap([0,1],[1,0]))
    image = img.lt(0)
    bands = image.select('B2').add(image.select('B3')).add(image.select('B4')).add(image.select('B5')).add(image.select('B6')).add(image.select('B7'))
    image_outlier = img.updateMask(bands.gt(0).rename('outlier'))
    return ee.ImageCollection([image_cloud,image_shadow,image_outlier]).sum()


# 图像可视化参数
visParams = {'bands': ['B5', 'B6', 'B4'],'min': 0,'max': 3000,'gamma': 1.4}

def invalid_pixels(img):
    validPixel = maskSR(img).select('pixel_qa').gt(0).rename('waterclass').float()
    invalidPixel = maskSR_reverse(img).select('pixel_qa').gt(0).remap([0,1],[1,0]).rename('waterclass').float()
    class_image = ee.ImageCollection([invalidPixel,validPixel]).sum()
    invalidPixels = class_image.eq(0).multiply(ee.Image.pixelArea()).divide(1e6)
    invalidarea = invalidPixels.reduceRegion(**{'reducer': ee.Reducer.sum(),'geometry': roi,'scale': 1000,'maxPixels': 1e14,'tileScale': 2}).get('waterclass')
    region = class_image.gte(0).multiply(ee.Image.pixelArea()).divide(1e6)
    regionarea = region.reduceRegion(**{'reducer': ee.Reducer.sum(),'geometry': roi,'scale': 1000,'maxPixels': 1e14,'tileScale': 2}).get('waterclass')
    rate = ee.Number(invalidarea).divide(ee.Number(regionarea)).multiply(100)
    return img.set({'invalid_rate':rate})

# 唯一的行列号

In [ ]:
for year in range(1999,2021,5):
    print(year)
    startDate = str(year) + '-01-01'
    endDate = str(year) + '-12-31'
    landsat5 = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR').filterBounds(roi).filterDate(startDate, endDate).map(invalid_pixels)  
    landsat7 = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR').filterBounds(roi).filterDate(startDate, endDate).map(invalid_pixels)           
    landsat8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR').filterBounds(roi).filterDate(startDate, endDate).map(invalid_pixels)
    # l8 = landsat8.merge(landsat7).merge(landsat5)
    landsat_image = landsat5.map(lambda i : i.setMulti({'path_row': ee.String(i.get('system:id')).slice(-15,-9)}))
    path_row = landsat_image.distinct('path_row').aggregate_array('path_row')
    print(path_row.getInfo())
    landsat_image = landsat7.map(lambda i : i.setMulti({'path_row': ee.String(i.get('system:id')).slice(-15,-9)}))
    path_row = landsat_image.distinct('path_row').aggregate_array('path_row')
    print(path_row.getInfo())
    landsat_image = landsat8.map(lambda i : i.setMulti({'path_row': ee.String(i.get('system:id')).slice(-15,-9)}))
    path_row = landsat_image.distinct('path_row').aggregate_array('path_row')
    print(path_row.getInfo())

In [ ]:
# ['122039', '122040', '123038', '123039']
# 显示边界范围
year = '2019'
startDate = year + '-01-01'
endDate = year + '-12-31'
landsat5 = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR').filterBounds(roi).filterDate(startDate, endDate)
landsat7 = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR').filterBounds(roi).filterDate(startDate, endDate)         
landsat8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR').filterBounds(roi).filterDate(startDate, endDate)
l8 = landsat8.merge(landsat7).merge(landsat5).map(lambda i : i.setMulti({'path_row': ee.String(i.get('system:id')).slice(-15,-9)}))
# '118027'需替换
q = l8.filter(ee.Filter.eq('path_row','123045'))
print(q.size().getInfo())
Map.addLayer(q.first().geometry(), {'color': 'green','opaticy':0.001}, "q")

# 选择影像

## 获取影像ID

In [ ]:
###### 加载行列号
### daqing
### ['119027', '119028', '120027', '120028'],'118028'
# path_row = ee.List(['119028', '120027', '120028'])
### tianjin
### ['122033', '123032', '123033'],'122032','124032','124033'
# path_row = ee.List(['122033', '123032', '123033'])
### wuhan
### ['122039', '123038', '123039']
# path_row = ee.List(['122039', '123038', '123039'])
### prd
### ['122044', '122045', '123044'],'121044','121045'
# path_row = ee.List(['122044', '122045', '123044'])
### tibet
### ['138035', '138036', '139035', '139036'],'140036','140035'
path_row = ee.List(['138035', '138036', '139035', '139036'])

## 时间
year = '2019'
startDate = year + '-01-01'
endDate = year + '-12-31'

In [ ]:
# 影像筛选函数
l5 = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR').filterBounds(roi).filterDate(startDate, endDate).map(invalid_pixels)  
l7 = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR').filterBounds(roi).filterDate(startDate, endDate).map(invalid_pixels)           
l8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR').filterBounds(roi).filterDate(startDate, endDate).map(invalid_pixels)
# def filter_image(id):
#     images_l8 = landsat8.map(lambda i:i.setMulti({'path_row': ee.String(i.get('system:id')).slice(-15,-9),'month':ee.Number.parse(ee.String(i.get('system:id')).slice(-4,-2))}))
#     month_l8 = images_l8.map(lambda i:i.set({'img_id':i.get('system:index')})).filter(ee.Filter.eq('path_row',id))
#     first_quarter_l8 = ee.ImageCollection(month_l8.filter(ee.Filter.lte('month',3)).filter(ee.Filter.gte('month',1)).sort('invalid_rate').first()).first()
#     second_quarter_l8 = ee.ImageCollection(month_l8.filter(ee.Filter.lte('month',6)).filter(ee.Filter.gte('month',4)).sort('invalid_rate').first()).first()
#     third_quarter_l8 = ee.ImageCollection(month_l8.filter(ee.Filter.lte('month',9)).filter(ee.Filter.gte('month',7)).sort('invalid_rate').first()).first()
#     fourth_quarter_l8 = ee.ImageCollection(month_l8.filter(ee.Filter.lte('month',12)).filter(ee.Filter.gte('month',10)).sort('invalid_rate').first()).first()
#     img_col = ee.ImageCollection([first_quarter_l8,second_quarter_l8,third_quarter_l8,fourth_quarter_l8])
#     return img_col
def filter_image(id):
    # l8
    images_l8 = l8.map(lambda i:i.setMulti({'path_row': ee.String(i.get('system:id')).slice(-15,-9),'month':ee.Number.parse(ee.String(i.get('system:id')).slice(-4,-2))}))
    month_l8 = images_l8.map(lambda i:i.set({'img_id':i.get('system:id')})).filter(ee.Filter.eq('path_row',id))
    first_quarter_l8 = ee.ImageCollection(month_l8.filter(ee.Filter.lte('month',3)).filter(ee.Filter.gte('month',1)).sort('invalid_rate')).first()
    second_quarter_l8 = ee.ImageCollection(month_l8.filter(ee.Filter.lte('month',6)).filter(ee.Filter.gte('month',4)).sort('invalid_rate')).first()
    third_quarter_l8 = ee.ImageCollection(month_l8.filter(ee.Filter.lte('month',9)).filter(ee.Filter.gte('month',7)).sort('invalid_rate')).first()
    fourth_quarter_l8 = ee.ImageCollection(month_l8.filter(ee.Filter.lte('month',12)).filter(ee.Filter.gte('month',10)).sort('invalid_rate')).first()
    # l7
    images_l7 = l7.map(lambda i:i.setMulti({'path_row': ee.String(i.get('system:id')).slice(-15,-9),'month':ee.Number.parse(ee.String(i.get('system:id')).slice(-4,-2))}))
    month_l7 = images_l7.map(lambda i:i.set({'img_id':i.get('system:id')})).filter(ee.Filter.eq('path_row',id))
    first_quarter_l7 = ee.ImageCollection(month_l7.filter(ee.Filter.lte('month',3)).filter(ee.Filter.gte('month',1)).sort('invalid_rate')).first()
    second_quarter_l7 = ee.ImageCollection(month_l7.filter(ee.Filter.lte('month',6)).filter(ee.Filter.gte('month',4)).sort('invalid_rate')).first()
    third_quarter_l7 = ee.ImageCollection(month_l7.filter(ee.Filter.lte('month',9)).filter(ee.Filter.gte('month',7)).sort('invalid_rate')).first()
    fourth_quarter_l7 = ee.ImageCollection(month_l7.filter(ee.Filter.lte('month',12)).filter(ee.Filter.gte('month',10)).sort('invalid_rate')).first()
    # l5
    images_l5 = l5.map(lambda i:i.setMulti({'path_row': ee.String(i.get('system:id')).slice(-15,-9),'month':ee.Number.parse(ee.String(i.get('system:id')).slice(-4,-2))}))
    month_l5 = images_l5.map(lambda i:i.set({'img_id':i.get('system:id')})).filter(ee.Filter.eq('path_row',id))
    first_quarter_l5 = ee.ImageCollection(month_l5.filter(ee.Filter.lte('month',3)).filter(ee.Filter.gte('month',1)).sort('invalid_rate')).first()
    second_quarter_l5 = ee.ImageCollection(month_l5.filter(ee.Filter.lte('month',6)).filter(ee.Filter.gte('month',4)).sort('invalid_rate')).first()
    third_quarter_l5 = ee.ImageCollection(month_l5.filter(ee.Filter.lte('month',9)).filter(ee.Filter.gte('month',7)).sort('invalid_rate')).first()
    fourth_quarter_l5 = ee.ImageCollection(month_l5.filter(ee.Filter.lte('month',12)).filter(ee.Filter.gte('month',10)).sort('invalid_rate')).first()
    img_col = ee.ImageCollection([first_quarter_l8,second_quarter_l8,third_quarter_l8,fourth_quarter_l8,first_quarter_l7,second_quarter_l7,third_quarter_l7,fourth_quarter_l7,first_quarter_l5,second_quarter_l5,third_quarter_l5,fourth_quarter_l5])
    return img_col

In [ ]:
## 计算在每个行列号上选择的月份和卫星类型
# landsat5 = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR').filterBounds(roi).filterDate(startDate, endDate).map(invalid_pixels)  
# landsat7 = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR').filterBounds(roi).filterDate(startDate, endDate).map(invalid_pixels)           
# landsat8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR').filterBounds(roi).filterDate(startDate, endDate).map(invalid_pixels)
# l8 = landsat8.merge(landsat7).merge(landsat5)
select_image = path_row.map(filter_image)
number = path_row.size().getInfo()
for i in range(0,number):
    print(ee.ImageCollection(select_image .get(i)).aggregate_array('month').getInfo())
    print(ee.ImageCollection(select_image .get(i)).aggregate_array('SATELLITE').getInfo())
    print(ee.ImageCollection(select_image .get(i)).aggregate_array('invalid_rate').getInfo())
    print(ee.ImageCollection(select_image .get(i)).aggregate_array('img_id').getInfo())

In [11]:
year = '1999'
startDate = year + '-01-01'
endDate = year + '-12-31'
landsat5 = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR').filterBounds(roi).filterDate(startDate, endDate)
landsat7 = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR').filterBounds(roi).filterDate(startDate, endDate)         
landsat8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR').filterBounds(roi).filterDate(startDate, endDate)
l8 = landsat8.merge(landsat7).merge(landsat5).map(lambda i : i.setMulti({'path_row': ee.String(i.get('system:id')).slice(-15,-9)}))
# '118027'需替换,'138035', '138036', '139035', '139036'
q = l8.filter(ee.Filter.eq('path_row','139035'))
Map.addLayer(q.first().geometry(), {'color': 'green','opaticy':0.001}, "q")
# q1 = l8.filter(ee.Filter.eq('path_row','119027'))
# Map.addLayer(q1.first().geometry(), {'color': 'green','opaticy':0.001}, "q1")

In [9]:
Map

Map(bottom=13252.0, center=[35.24374500727302, 90.62259999999999], controls=(WidgetControl(options=['position'…

In [ ]:
## 合并在各个行列号上选择的影像
img_col = ee.ImageCollection([])
for i in range(0,number):
    img_col = img_col.merge(ee.ImageCollection(select_image.get(i)))
sample_image = img_col
print(sample_image.size().getInfo())
print(sample_image.aggregate_array('month').getInfo())
print(sample_image.aggregate_array('img_id').getInfo())

## 加载样本集

In [ ]:
region = 'wuhan'
sample_region = 'WUHAN'
year = '2009'
## 永久性水体样本和非水体样本，抽样
pw_nw = ee.FeatureCollection('users/311605001111/SAMPLE_' + sample_region + '/' + region + year)
permanent_water = pw_nw.filter(ee.Filter.eq('point_type','permanent_water'))
no_water = pw_nw.filter(ee.Filter.eq('point_type','no_water'))
season_water = pw_nw.filter(ee.Filter.eq('point_type','season'))

# def filter_sample_pw(id):
#     pw_sample = pepermanent_water.filter(ee.Filter.eq('img_id',id))
#     nw_sample = no_water.filter(ee.Filter.eq('img_id',id))
    
# def filter_sample_nw(id):
#     pw_sample = pepermanent_water.filter(ee.Filter.eq('img_id',id))
#     nw_sample = no_water.filter(ee.Filter.eq('img_id',id))

# def filter_sample_sw(id):
#     pw_sample = pepermanent_water.filter(ee.Filter.eq('img_id',id))
#     nw_sample = no_water.filter(ee.Filter.eq('img_id',id))

In [ ]:
print(pw_nw.first().getInfo())

In [ ]:
zz = sample_image.aggregate_array('img_id').map(lambda id : pw_nw.filter(ee.Filter.eq('Image_id',id))).flatten()
print(zz.flatten().size().getInfo())
img_col = ee.ImageCollection([])
number = sample_image.size().getInfo()
for i in range(0,number):
    img_col = img_col.merge(ee.FeatureCollection(zz.get(i)))
print(img_col.size().getInfo())

In [ ]:
zz = sample_image.aggregate_array('img_id').map(lambda id : permanent_water.filter(ee.Filter.eq('Image_id',id))).flatten()
print(zz.flatten().size().getInfo())
img_col = ee.ImageCollection([])
number = sample_image.size().getInfo()
for i in range(0,number):
    img_col = img_col.merge(ee.FeatureCollection(zz.get(i)))
print(img_col.size().getInfo())

In [ ]:
zz = sample_image.aggregate_array('img_id').map(lambda id : no_water.filter(ee.Filter.eq('Image_id',id))).flatten()
print(zz.flatten().size().getInfo())
img_col = ee.ImageCollection([])
number = sample_image.size().getInfo()
for i in range(0,number):
    img_col = img_col.merge(ee.FeatureCollection(zz.get(i)))
print(img_col.size().getInfo())

In [ ]:
zz = sample_image.aggregate_array('img_id').map(lambda id : season_water.filter(ee.Filter.eq('Image_id',id))).flatten()
print(zz.flatten().size().getInfo())
img_col = ee.ImageCollection([])
number = sample_image.size().getInfo()
for i in range(0,number):
    img_col = img_col.merge(ee.FeatureCollection(zz.get(i)))
print(img_col.size().getInfo())

In [ ]:
zz = sample_image.aggregate_array('img_id').map(lambda id : season_water.filter(ee.Filter.eq('Image_id',id))).flatten()
print(zz.size().getInfo())

In [ ]:
region = 'wuhan'
sample_region = 'WUHAN'
year = '2019'
## 永久性水体样本和非水体样本，抽样
pw_nw = ee.FeatureCollection('users/311605001111/SAMPLE_' + sample_region + '/' + region + year)
print(pw_nw.size().getInfo())
permanent_water = pw_nw.filter(ee.Filter.eq('waterclass',1))
no_water = pw_nw.filter(ee.Filter.eq('waterclass',0))
print(pw_nw.first().getInfo())

In [ ]:
permanent_water = pw_nw.filter(ee.Filter.eq('point_type','permanent_water')).filter(ee.Filter.eq('Image_id',id))
no_water = pw_nw.filter(ee.Filter.eq('point_type','no_water')).filter(ee.Filter.eq('Image_id',id))
season_water = pw_nw.filter(ee.Filter.eq('point_type','season')).filter(ee.Filter.eq('Image_id',id))

In [ ]:
# 影像筛选函数
def filter_image(id):
    # l8
    images_l8 = l8.map(lambda i:i.setMulti({'path_row': ee.String(i.get('system:id')).slice(-15,-9),'month':ee.Number.parse(ee.String(i.get('system:id')).slice(-4,-2))}))
    month_l8 = images_l8.map(lambda i:i.set({'img_id':i.get('system:index')})).filter(ee.Filter.eq('path_row',id))
    first_quarter_l8 = ee.ImageCollection(month_l8.filter(ee.Filter.lte('month',3)).filter(ee.Filter.gte('month',1)).randomColumn('random',1,'uniform').sort('random')).first()
    second_quarter_l8 = ee.ImageCollection(month_l8.filter(ee.Filter.lte('month',6)).filter(ee.Filter.gte('month',4)).randomColumn('random',1,'uniform').sort('random')).first()
    third_quarter_l8 = ee.ImageCollection(month_l8.filter(ee.Filter.lte('month',9)).filter(ee.Filter.gte('month',7)).randomColumn('random',1,'uniform').sort('random')).first()
    fourth_quarter_l8 = ee.ImageCollection(month_l8.filter(ee.Filter.lte('month',12)).filter(ee.Filter.gte('month',10)).randomColumn('random',1,'uniform').sort('random')).first()
    # l7
    images_l7 = l7.map(lambda i:i.setMulti({'path_row': ee.String(i.get('system:id')).slice(-15,-9),'month':ee.Number.parse(ee.String(i.get('system:id')).slice(-4,-2))}))
    month_l7 = images_l7.map(lambda i:i.set({'img_id':i.get('system:index')})).filter(ee.Filter.eq('path_row',id))
    first_quarter_l7 = ee.ImageCollection(month_l7.filter(ee.Filter.lte('month',3)).filter(ee.Filter.gte('month',1)).randomColumn('random',1,'uniform').sort('random')).first()
    second_quarter_l7 = ee.ImageCollection(month_l7.filter(ee.Filter.lte('month',6)).filter(ee.Filter.gte('month',4)).randomColumn('random',1,'uniform').sort('random')).first()
    third_quarter_l7 = ee.ImageCollection(month_l7.filter(ee.Filter.lte('month',9)).filter(ee.Filter.gte('month',7)).randomColumn('random',1,'uniform').sort('random')).first()
    fourth_quarter_l7 = ee.ImageCollection(month_l7.filter(ee.Filter.lte('month',12)).filter(ee.Filter.gte('month',10)).randomColumn('random',1,'uniform').sort('random')).first()
    # l5
    images_l5 = l5.map(lambda i:i.setMulti({'path_row': ee.String(i.get('system:id')).slice(-15,-9),'month':ee.Number.parse(ee.String(i.get('system:id')).slice(-4,-2))}))
    month_l5 = images_l5.map(lambda i:i.set({'img_id':i.get('system:index')})).filter(ee.Filter.eq('path_row',id))
    first_quarter_l5 = ee.ImageCollection(month_l5.filter(ee.Filter.lte('month',3)).filter(ee.Filter.gte('month',1)).randomColumn('random',1,'uniform').sort('random')).first()
    second_quarter_l5 = ee.ImageCollection(month_l5.filter(ee.Filter.lte('month',6)).filter(ee.Filter.gte('month',4)).randomColumn('random',1,'uniform').sort('random')).first()
    third_quarter_l5 = ee.ImageCollection(month_l5.filter(ee.Filter.lte('month',9)).filter(ee.Filter.gte('month',7)).randomColumn('random',1,'uniform').sort('random')).first()
    fourth_quarter_l5 = ee.ImageCollection(month_l5.filter(ee.Filter.lte('month',12)).filter(ee.Filter.gte('month',10)).randomColumn('random',1,'uniform').sort('random')).first()
    img_col = ee.ImageCollection([first_quarter_l8,second_quarter_l8,third_quarter_l8,fourth_quarter_l8,first_quarter_l7,second_quarter_l7,third_quarter_l7,fourth_quarter_l7,first_quarter_l5,second_quarter_l5,third_quarter_l5,fourth_quarter_l5])
    return img_col

## 计算唯一的行列号
l5 = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR').filterBounds(roi).filterDate(startDate, endDate)  
l7 = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR').filterBounds(roi).filterDate(startDate, endDate)           
l8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR').filterBounds(roi).filterDate(startDate, endDate)
region_images = l8.merge(l7).merge(l5).map(filter_image)
landsat_image = region_images.map(lambda i : i.setMulti({'path_row': ee.String(i.get('system:id')).slice(-15,-9),'month':ee.Number.parse(ee.String(i.get('system:id')).slice(-4,-2))}))
path_row = landsat_image.distinct('path_row').aggregate_array('path_row')
print(path_row.getInfo())

## 计算在每个行列号上选择的月份和卫星类型
select_image = path_row.map(filter_image)
number = path_row.size().getInfo()
print(path_row.size().getInfo())
for i in range(0,number):
    print(ee.ImageCollection(select_image .get(i)).aggregate_array('month').getInfo())
    print(ee.ImageCollection(select_image .get(i)).aggregate_array('SATELLITE').getInfo())

In [ ]:
## 合并在各个行列号上选择的影像
img_col = ee.ImageCollection([])
for i in range(0,number):
    img_col = img_col.merge(ee.ImageCollection(select_image.get(i)))
sample_image = img_col.map(water_index).map(maskSR)
print(sample_image.size().getInfo())
print(sample_image.aggregate_array('path_row').getInfo())
print(sample_image.aggregate_array('month').getInfo())